### Trabalho Pratico NLP 2021/2
##### Nome: Guilherme Cramer 
##### Matricula: 2021666152

#### Treinamento do modelo de Linguagem. 
Primeiramente vamos importar os pacotes necessários. Será usado a biblioteca "gensin.Word2vec" para o modelo de linguagem (Cbow e Skip-gran). 

A biblioteca "spacy" será usada para o preprocessamento do texto. No preprocessamento, foi escolhido 4 formas de tratamento. A primeira transforma a palavram em minúscula; a segunda elimina as palavras que não tem significado semântico, como palavras de conexões e artigos ("the" , "a",... etc). A terceira elimina as pontuações; já o quarto processamento realiza a "lemmarização" da palavra, ou seja, transforma ela em sua forma mais original. Por exemplo transforma "working" em "work". 

In [8]:
import gensim
import pandas as pd
import spacy
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
nlp = spacy.load("en_core_web_sm")


In [9]:

def cleanData(doc,stemming = False):
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return " ".join(final_token)


Leitura dos dados sem tratamentos. 

In [16]:
data_path = 'C:\\Users\\gcram\\Documents\\Datasets\\text8\\text8'
with open(data_path) as f:
    lines = f.readlines()
data = lines[0]

Como o tamanho do dataset é muito grande para o modelo em ingles do "spacy", o preprocessamento foi feito em partes. Mas não altera em nada.  

In [17]:
data_s = data.split(' ')
N = len(data_s)
bs = 100000
n_batch = int(len(data_s)/bs) + 1 


In [19]:
ini = 0
res = ''
for batch in range(n_batch):
    end = ini + bs
    if end < N:
        data_bat = data[ini:end]
    else:
        data_bat = data[ini:]
    d = ''.join(data_bat)
    d_procss = cleanData(d)
    res = res + ' ' + ''.join(d_procss)
    

Salva os dados já processados.

In [20]:
data_save = 'C:\\Users\\gcram\\Documents\\Datasets\\text8\\text8_preprocss'
with open(data_save, 'w') as f:
    f.write(res)

Carrega os dados para serem passados para o modelo treinar.

In [21]:
data = Text8Corpus('C:\\Users\\gcram\\Documents\\Datasets\\text8\\text8_preprocss')

São criados e treinados algums modelos variando-se os parâmetros de tamanho da janela e também o tamanho do vetor de representação. Apesar de nas instruções do Trabalho ter pedido para variar o "training size", eu não entendi o que seria isso. Creio que a variação do tamanho do vetor de representação também seja bem interessante. 
Após o treinamento o modelo é salvo localmente para ser avaliado no outro código. 


In [22]:
import os,glob
os.chdir("C:\\Users\\gcram\\Documents\\GitHub\\NLP\\LanguageModel\\")
models = glob.glob("*.model")

In [24]:
d = {}
d['skip-gram'] = 1
d['cbow'] = 0

for ws in [5,10,20,100]:
    for ts in [20,50,100,200]:
        for method in ['skip-gram','cbow']:
            model_name = f"word2vec{ws}_{ts}_{method}.model"
            if model_name in models:
                continue
            model = Word2Vec(sentences=data, 
                 vector_size=ts,
                 window=ws, 
                 min_count=1, 
                 workers=4,
                sg=d[method])
            model.save(model_name)
            

#### Referências: 
https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus
https://www.kaggle.com/theainerd/beginners-s-guide-to-nlp-using-spacy